In [2]:
from batchgenerators.utilities.file_and_folder_operations import *
import cv2
import os
from skimage.filters import threshold_otsu
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json

Setting up nnU-Net folders

In [8]:
ct_dir:str = '/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/CAT_scans_Preprocessed',
nnunet_dataset_id: int = 501
task_name = "Skull"

foldername = "Dataset%03.0d_%s" % (nnunet_dataset_id, task_name)
nnUNet_raw = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw"
nnUNet_preprocessed = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_preprocessed"
out_base = join(nnUNet_raw, foldername)

Process all images from Subset3 and store them in nnUnet_preprocessed folder



Generate .json file

In [9]:

generate_dataset_json(out_base,
                      channel_names={0: "CT"},
                      labels={
                          "background": 0,
                          "skull": 1
                          },
                        num_training_cases=576, 
                        file_ending='.png',
                        dataset_name=task_name, 
                        reference='https://github.com/BYO-UPM/Craneal_CT',
                        release='release',
                        description=task_name)

In [14]:
import os
import cv2
import numpy as np

input_dir = '/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/Dataset/Subset3'
ind = [501]

fixed_threshold = 127  # Set your desired threshold value here

for i in ind:
    output_dir = f'/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset{i}_Skull/imagesTr'
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    counter = 1

    # Walk through all subdirectories and files in input_dir
    for root, dirs, files in os.walk(input_dir):
        for filename in sorted(files):
            if filename.endswith('.png'):
                # Construct the full path to the image
                img_path = os.path.join(root, filename)

                # Open the image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Save the binary image
                new_filename = f"10_{counter:02d}.png"
                output_path = os.path.join(output_dir, new_filename)
                cv2.imwrite(output_path, img)

                counter += 1


Get the binary masks: range (0, 1)

In [16]:
input_dir = '/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset501_Skull/labelsTr'
ind = [0]
for i in ind:
    output_dir = f'/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset501_Skull/labelsTr'
    counter = 1
    for filename in sorted(os.listdir(input_dir)):
        if filename.endswith('.png'):
            # Open the image
            img_path = os.path.join(input_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
            # Apply threshold to create binary mask
            threshold1 = threshold_otsu(img)
            binary_image = img > threshold1
            binary_image=binary_image.astype(int)
                
            # Save the binary image
            new_filename = f"10_{counter:02d}.png"
            counter += 1
            output_path = os.path.join(output_dir, new_filename)
            cv2.imwrite(output_path, binary_image)

Change all the CT images into 1 channel grey-scale images

In [15]:
def convert_to_grayscale_cv2(source_directory, destination_directory):
    # Initialize a counter for the filenames
    counter = 1

    # Iterate over all files in the source directory
    for filename in sorted(os.listdir(source_directory)):
        if filename.endswith(".png"):
            # Read the image in grayscale mode
            img = cv2.imread(os.path.join(source_directory, filename), cv2.IMREAD_GRAYSCALE)
            # Save the grayscale image to the destination directory
            new_filename = f"10_{counter:02d}_0000.png"
            counter += 1
            cv2.imwrite(os.path.join(destination_directory, new_filename), img)
            # Remove the original image
            os.remove(os.path.join(source_directory, filename))
            
            # Verify the image has 1 channel
            if len(img.shape) == 2:
                print(f"{filename} is successfully converted to grayscale with 1 channel.")
            else:
                print(f"Error: {filename} still has {img.shape[-1]} channels.")
            
            print(f"Converted {filename} to grayscale and saved.")

source_dir = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset501_Skull/imagesTr"
dest_dir = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset501_Skull/imagesTr"
convert_to_grayscale_cv2(source_dir, dest_dir)

# Remove all that doesnt end in 0000.png


10_01.png is successfully converted to grayscale with 1 channel.
Converted 10_01.png to grayscale and saved.
10_02.png is successfully converted to grayscale with 1 channel.
Converted 10_02.png to grayscale and saved.
10_03.png is successfully converted to grayscale with 1 channel.
Converted 10_03.png to grayscale and saved.
10_04.png is successfully converted to grayscale with 1 channel.
Converted 10_04.png to grayscale and saved.
10_05.png is successfully converted to grayscale with 1 channel.
Converted 10_05.png to grayscale and saved.
10_06.png is successfully converted to grayscale with 1 channel.
Converted 10_06.png to grayscale and saved.
10_07.png is successfully converted to grayscale with 1 channel.
Converted 10_07.png to grayscale and saved.
10_08.png is successfully converted to grayscale with 1 channel.
Converted 10_08.png to grayscale and saved.
10_09.png is successfully converted to grayscale with 1 channel.
Converted 10_09.png to grayscale and saved.
10_10.png is succes